In [2]:
import kagglehub
import pandas as pd
import plotly.graph_objects as go

# Download data
path = kagglehub.dataset_download("rohanrao/nifty50-stock-market-data")

files_f = ['HDFCBANK.csv', 'ICICIBANK.csv', 'SBIN.csv', 'KOTAKBANK.csv']
files_t = ['INFY.csv', 'TCS.csv', 'HCLTECH.csv', 'WIPRO.csv']
files_e = ['RELIANCE.csv', 'ONGC.csv', 'NTPC.csv', 'POWERGRID.csv']

dff = pd.DataFrame()
dft = pd.DataFrame()
dfe = pd.DataFrame()

for x in files_f:
    df_f = pd.read_csv(f"{path}/{x}")
    dff = pd.concat([dff, df_f], ignore_index=True)

for x in files_t:
    df_t = pd.read_csv(f"{path}/{x}")
    dft = pd.concat([dft, df_t], ignore_index=True)

for x in files_e:
    df_e = pd.read_csv(f"{path}/{x}")
    dfe = pd.concat([dfe, df_e], ignore_index=True)

dff['Sector'] = 'Financial'
dft['Sector'] = 'Technology'
dfe['Sector'] = 'Energy'

final = pd.concat([dff, dft, dfe], ignore_index=True)
final['Date'] = pd.to_datetime(final['Date'])

final = final.sort_values(['Sector', 'Symbol', 'Date'])
final['Daily_R'] = final.groupby(['Sector', 'Symbol'])['Close'].pct_change()

# Calculate rolling volatility (60-day) by sector
df_sector_vol = final.groupby(['Date', 'Sector'])['Daily_R'].std().reset_index()
df_sector_vol.columns = ['Date', 'Sector', 'Volatility']
df_sector_vol['Volatility'] = df_sector_vol['Volatility'] * 100  # Convert to percentage

df_sector_vol = df_sector_vol.sort_values(['Sector', 'Date'])

# Resample to monthly for smoother chart
df_plot = (df_sector_vol.set_index('Date')
           .groupby('Sector')['Volatility']
           .resample('ME').mean()
           .reset_index())

# Create Plotly chart
colors = {
    'Financial': '#1f77b4',
    'Technology': '#ff7f0e',
    'Energy': '#2ca02c'
}

fig = go.Figure()

for sector in df_plot['Sector'].unique():
    sector_df = df_plot[df_plot['Sector'] == sector]

    hover = [
        f"<b>{sector}</b><br>" +
        f"Date: {row['Date'].strftime('%b %Y')}<br>" +
        f"Volatility: {row['Volatility']:.2f}%"
        for _, row in sector_df.iterrows()
    ]

    fig.add_trace(go.Scatter(
        x=sector_df['Date'],
        y=sector_df['Volatility'],
        mode='lines',
        name=sector,
        line=dict(color=colors[sector], width=2),
        hovertemplate='%{customdata}<extra></extra>',
        customdata=hover
    ))

# Add crisis shading
fig.add_vrect(x0="2008-06-01", x1="2009-06-01",
              fillcolor="rgba(255,0,0,0.1)", line_width=0,
              annotation_text="2008 Crisis", annotation_position="top left",
              annotation_font=dict(color="red", size=11))

fig.add_vrect(x0="2020-02-01", x1="2020-08-01",
              fillcolor="rgba(128,0,128,0.1)", line_width=0,
              annotation_text="COVID-19", annotation_position="top left",
              annotation_font=dict(color="purple", size=11))

fig.update_layout(
    title=dict(
        text='<b>Sector Volatility Over Time</b>',
        font=dict(size=18),
        x=0.5
    ),
    xaxis=dict(
        title='Date',
        rangeselector=dict(
            buttons=[
                dict(count=5, label='5Y', step='year', stepmode='backward'),
                dict(count=10, label='10Y', step='year', stepmode='backward'),
                dict(step='all', label='All')
            ],
            x=0, y=1.12
        )
    ),
    yaxis=dict(title='Volatility (%)'),
    template='plotly_white',
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
    height=500
)

fig.write_html("sector_volatility.html", include_plotlyjs='cdn')
fig.show()
print("Saved: sector_volatility.html")

Using Colab cache for faster access to the 'nifty50-stock-market-data' dataset.


Saved: sector_volatility.html
